### About this result

Analyzing the results runnning on IBM quantum platform, the result data is stored in `./pred_outputs/ibmq` with `npy` format.

In [2]:
import os
from itertools import product

import numpy as np

import g_main

general_config = g_main.json_config
ibmq_dir = os.path.join(general_config["predictions_dir"], "ibmq")

#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [31]:
shots = 1024
batch_size = 10
num_pt_ptcs = 4
num_bin_data = 25
backend_list = ["ibm_cusco"]
channel_list = ["BB-QCD", "TT-QCD"]

def get_ibmq_output(npy_path):
    result = np.load(npy_path, allow_pickle=True)
    result = result.item()
    return result

rnd_seed= 0
for abbrev, backend in product(channel_list, backend_list):
    print(f"* Channel: {abbrev} | Backend: {backend} | Size: {len(y_true)}\n")
    print(f"      | SIM  ACC | IBMQ ACC | S/I  ACC | S/I DIFF")
    for Q in [3, 6, 9]:
        # Horizontal splitting line.
        print(f"--------------------------------------------------")
        
        # Read npy file.
        npy_file = (
            f"{backend}-s{shots}-{abbrev}_"
            + f"r{rnd_seed}-p{num_pt_ptcs}_"
            + f"n{num_bin_data}_"
            + f"b{batch_size}-{Q}_1_{Q}"
            + ".npy"
        )
        result = np.load(os.path.join(ibmq_dir, npy_file), allow_pickle=True)
        result = result.item()

        # Prepare true labels and predictions.
        y_true = result["y_true"].reshape(-1).numpy()
        y_penl = result["y_penl"].reshape(-1).numpy()
        y_ibmq = result["y_ibmq"].reshape(-1).numpy()

        # Calculate accuracy and loss (difference).
        SIM__ACC = (y_true == (y_penl > 0)).mean()
        IBMQ_ACC = (y_true == (y_ibmq > 0)).mean()
        S_I__ACC = ((y_penl > 0) == (y_ibmq > 0)).mean()
        S_I_DIFF = np.mean(np.abs(y_penl - y_ibmq))
        print(f"Q = {Q} |   {SIM__ACC:.2f}   |   {IBMQ_ACC:.2f}   |   {S_I__ACC:.2f}   |   {S_I_DIFF:.2f}   ")
    print("\n" + "#"*50 + "\n")

* Channel: BB-QCD | Backend: ibm_cusco | Size: 100

      | SIM  ACC | IBMQ ACC | S/I  ACC | S/I DIFF
--------------------------------------------------
Q = 3 |   0.52   |   0.50   |   0.98   |   0.28   
--------------------------------------------------
Q = 6 |   0.54   |   0.50   |   0.86   |   0.46   
--------------------------------------------------
Q = 9 |   0.62   |   0.52   |   0.80   |   0.51   

##################################################

* Channel: TT-QCD | Backend: ibm_cusco | Size: 100

      | SIM  ACC | IBMQ ACC | S/I  ACC | S/I DIFF
--------------------------------------------------
Q = 3 |   0.58   |   0.50   |   0.86   |   0.47   
--------------------------------------------------
Q = 6 |   0.81   |   0.52   |   0.49   |   0.71   
--------------------------------------------------
Q = 9 |   0.64   |   0.41   |   0.59   |   0.73   

##################################################

